In [3]:
import os

from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.chat import HumanMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda


load_dotenv()

llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
    api_key=os.getenv('OPENAI_API_KEY'),
    max_tokens=4096,
)


def taste_food(x):
    prompt = ChatPromptTemplate.from_messages([
        ("system", """When provided with an input question, please identify the dish in question and describe what it tastes like."""),
    ])

    template = []
    if x['image_urls']:
        template += [{'image_url': {'url': image_url}} for image_url in x['image_urls']]
    template += [{'text': x['query']}]
    prompt += HumanMessagePromptTemplate.from_template(template=template)

    return prompt | llm | StrOutputParser()


def recommend_wine(x):
    prompt = ChatPromptTemplate.from_messages([
        ("system", """
You are ChatGPT, a professional sommelier who has gone through a rigorous training process step by step, driven by a deep curiosity about wine. You possess a keen sense of smell, a keen sense of exploration, and an awareness of the many details in wine.

Your task is to accurately identify an appropriate wine pairing based on the provided food description in triple quotes. 대답은 한글로 해줘

### Task Instructions:

1. **Food Analysis**:
- Carefully read the food description provided within triple quotes.
- Identify key characteristics of the food, including flavors, textures, cooking methods, and any prominent ingredients.

2. **Wine Review Analysis**:
- Carefully read the wine review and metadata provided within triple quotes.
- Identify key characteristics of the wine, including aroma, flavor, tannin structure, acidity, body, and finish.

3. **Pairing Recommendation**:
- Recommend a specific wine (including grape variety, region of origin, and possible vintage) that pairs well with the described food.
- Explain why this wine is a suitable match for the food, taking into account factors such as acidity, tannin structure, body, and flavor profile.

### Example:

**Food**:
```Triple-cooked pork belly with a crispy skin, served with a tangy apple and fennel slaw, and a rich, savory jus.```

**Wine Review**:
```
winery: Barossa Valley Shiraz
description: This full-bodied Shiraz from Barossa Valley, Australia, boasts rich flavors of dark berries, black pepper, and a hint of vanilla. It has a robust tannin structure and a long, spicy finish.
points: 92
price: 30
variety: Shiraz
```

**Wine Pairing Recommendation**:
```A full-bodied red wine such as a Shiraz from Barossa Valley, Australia, would pair excellently with the triple-cooked pork belly. The wine’s robust tannins and dark fruit flavors will complement the rich, savory notes of the pork, while its hint of spice will enhance the tangy apple and fennel slaw. The acidity in the Shiraz will cut through the fat of the pork, creating a balanced and harmonious dining experience.```

### Now, provide a food description in triple quotes for the system to recommend a wine pairing:
**Food**:
```
{food}
```

**Wine Review**:
```
{reviews}
```
"""),
    ])
    return prompt | llm | StrOutputParser()


def recommend_food(x):
    prompt = ChatPromptTemplate.from_messages([
        ("system", """
You are ChatGPT, a professional sommelier who has gone through a rigorous training process step by step, driven by a deep curiosity about wine. You possess a keen sense of smell, a keen sense of exploration, and an awareness of the many details in wine.

Your task is to accurately identify the wine in question and recommend an appropriate food pairing based on the provided wine reviews in triple quotes. 대답은 한글로 해줘

### Task Instructions:

1. **Review Analysis**:
- Carefully read the wine review provided within triple quotes.
- Identify key characteristics of the wine, including aroma, flavor, tannin structure, acidity, body, and finish.

2. **Wine Identification**:
- Based on the identified characteristics, determine the specific type of wine, its possible grape variety, region of origin, and vintage if possible.
- Consider elements such as fruit notes (e.g., cherry, blackberry), non-fruit notes (e.g., oak, vanilla), and structural components (e.g., tannins, acidity).

3. **Pairing Recommendation**:
- Recommend an appropriate food pairing that complements the identified wine.
- Take into account the wine’s characteristics (e.g., a full-bodied red wine with high tannins pairs well with rich, fatty meats).
- Suggest alternative pairings such as cheese, desserts, or even specific dishes that enhance the wine's profile.
    """),
    ])

    template = []
    if x['image_urls']:
        template += [{'image_url': {'url': image_url}} for image_url in x['image_urls']]
    template += [{'text': x['query']}]
    prompt += HumanMessagePromptTemplate.from_template(template=template)

    return prompt | llm | StrOutputParser()

In [4]:
recommend_food("샤도네이")

TypeError: string indices must be integers, not 'str'

In [4]:
import os

from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import HumanMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda



load_dotenv(

)

llm = ChatOpenAI(
    model="gemini-pro",
    temperature=0,
    api_key=os.getenv('OPENAI_API_KEY'),
    max_tokens=4096,
)

In [5]:
llm.invoke("안녕하세요")

NotFoundError: Error code: 404 - {'error': {'message': 'The model `gemini-pro` does not exist or you do not have access to it.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}

In [ ]:
#function_call
"""
https://platform.openai.com/docs/guides/function-calling

"""
import os

from langchain_openai import ChatOpenAI
from langchain.globals import set_verbose, set_debug
from dotenv import load_dotenv

load_dotenv()

set_debug(True)
set_verbose(True)

# 환경 변수 읽기
llm = ChatOpenAI(model='gpt-4o', api_key=os.getenv('OPENAI_API_KEY'), temperature=0, max_tokens=4096)

json_schema = {
    "title": "joke",
    "description": "Joke to tell user.",
    "type": "object",
    "properties": {
        "setup": {
            "type": "string",
            "description": "The setup of the joke",
        },
        "punchline": {
            "type": "string",
            "description": "The punchline to the joke",
        },
        "rating": {
            "type": "integer",
            "description": "How funny the joke is, from 1 to 10",
        },
    },
    "required": ["setup", "punchline"],
}
structured_llm = llm.with_structured_output(json_schema)

structured_llm.invoke("Tell me a joke about cats")

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "input": "Tell me a joke about cats"
}
[llm/start] [chain:RunnableSequence > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: Tell me a joke about cats"
  ]
}
[llm/end] [chain:RunnableSequence > llm:ChatOpenAI] [1.27s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "{\"setup\":\"Why was the cat sitting on the computer?\",\"punchline\":\"Because it wanted to keep an eye on the mouse!\",\"rating\":7}",
            "additional_kwargs": {
              "parsed": null,
              "refusal": null
     

{'setup': 'Why was the cat sitting on the computer?',
 'punchline': 'Because it wanted to keep an eye on the mouse!',
 'rating': 7}

In [7]:
print(structured_llm)

first=RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x00000141A20E0250>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x00000141A2090D50>, root_client=<openai.OpenAI object at 0x00000141A11D6F50>, root_async_client=<openai.AsyncOpenAI object at 0x00000141A11CE690>, model_name='gpt-4o', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'), max_tokens=4096), kwargs={'response_format': {'type': 'json_schema', 'json_schema': {'name': 'joke', 'description': 'Joke to tell user.', 'strict': False, 'schema': {'type': 'object', 'properties': {'setup': {'type': 'string', 'description': 'The setup of the joke'}, 'punchline': {'type': 'string', 'description': 'The punchline to the joke'}, 'rating': {'type': 'integer', 'description': 'How funny the joke is, from 1 to 10'}}, 'required': ['setup', 'punchline']}}}, 'ls_structured_output_format': {'kwargs': {'method': 'json_schem